# Discussion

This has stats used in the discussion

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
import rpy2
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')

%load_ext rpy2.ipython
%matplotlib inline

## Getting the size of a button in the middle of the screen

In [2]:
def get_intersect_from_pts(x1, y1, x2, y2):
    if x1 == x2:
        return x1
    m = (y2 - y1) / (x2 - x1)
    b = y1 - m * x1
    return -b / m # get the x-intercept

def get_intersect_y(x):
    x = x.sort_values("time")
    for i in range(len(x) - 1):
        x1 = x["position_x"].iloc[i]
        y1 = x["position_y"].iloc[i]
        x2 = x["position_x"].iloc[i + 1]
        y2 = x["position_y"].iloc[i + 1]
        if (y1 >= 0 and y2 < 0) or (y1 <= 0 and y2 > 0):
            return get_intersect_from_pts(x1, y1, x2, y2)

traj_df = pd.read_csv("../data/trajectories.csv")
traj_df = traj_df[traj_df["task"].isin([5, 11, 17, 23])]
a = traj_df.groupby(["participant", "prototype", "input_type", "is_long", "block", "task"]).apply(get_intersect_y)
a = a.to_frame(name="yint")
a.reset_index(inplace=True)
a

,participant,prototype,input_type,is_long,block,task,yint
0,0,12mm,pen,False,0,17,-0.390159
1,0,12mm,pen,False,0,23,-0.485655
2,0,12mm,pen,False,1,17,-0.272673
3,0,12mm,pen,False,1,23,-0.416660
4,0,12mm,pen,False,2,17,-0.187191
...,...,...,...,...,...,...,...
1735,14,x1fold,touch,False,0,11,0.058833
1736,14,x1fold,touch,False,1,5,-0.026077
1737,14,x1fold,touch,False,1,11,0.303508
1738,14,x1fold,touch,False,2,5,0.024979


In [3]:
std = a[a["is_long"] == False].groupby(["input_type"]).std()["yint"]
print("Size of target for 95% success:", 4 * std)
mean = a[a["is_long"] == False].groupby(["input_type"]).mean()["yint"]

min_pen = mean["pen"] - 2 * std["pen"]
max_pen = mean["pen"] + 2 * std["pen"]

min_touch = mean["touch"] - 2 * std["touch"]
max_touch = mean["touch"] + 2 * std["touch"]

pen_tasks = a[(a["is_long"] == False) & (a["input_type"] == "pen")]
pen_n_in = len(pen_tasks[(pen_tasks["yint"] > min_pen) & (pen_tasks["yint"] < max_pen)])
pen_n_tot = len(pen_tasks)
print("% of drags in bounds for pen:", pen_n_in / pen_n_tot)

touch_tasks = a[(a["is_long"] == False) & (a["input_type"] == "touch")]
touch_n_in = len(touch_tasks[(touch_tasks["yint"] > min_touch) & (touch_tasks["yint"] < max_touch)])
touch_n_tot = len(touch_tasks)
print("% of drags in bounds for touch:", touch_n_in / touch_n_tot)

Size of target for 95% success: input_type
pen      1.216544
touch    1.254909
Name: yint, dtype: float64
% of drags in bounds for pen: 0.95625
% of drags in bounds for touch: 0.9458333333333333
